# Dependency setup

## Package setup

In [57]:
# Import dependencies and libraries
from IPython.display import clear_output
import pandas as pd
from census import Census
from us import states
from dotenv import load_dotenv
import requests
import sys
import csv
import os
import json
import time

# Load development mode
try:
  dev = os.getenv("DEV_MODE")
  if dev: print("Development mode loaded successfully!")
except:
  print("Error loading development mode")
  print("Defaulting to production mode")
  dev = False

# Load environment variables and U.S. Census API key
if dev: print("Loading environment variables...")
load_dotenv()
if dev: print("Environment variables loaded successfully!")
try:
  if dev: print("Loading U.S. Census API key...")
  api_key = os.getenv("CENSUS_API_KEY")
  if dev: print("API key loaded successfully!")
except:
  print("Error loading U.S. Census API key")
  print("Please check your environment variables")
  

Development mode loaded successfully!
Loading environment variables...
Environment variables loaded successfully!
Loading U.S. Census API key...
API key loaded successfully!


## Google Colab setup

In [ ]:
# Install the required libraries for running on colab
try:
  %pip install prophet
except:
  print("Error installing libraries to colab instance")
finally:
  clear_output()
  print("Libraries successfully installed to colab instance")


# Census Code

In [58]:
# Create an instance of the Census library
try:
  if dev: print("Creating an instance of the Census API...")
  c = Census(
      api_key
  )
  if dev: print("Census API instance created:")
  if dev: display(type(c))
  if dev: display(c)
except:
  print("Error creating an instance of the Census API")
  print("Please check your U.S. Census API key")


Creating an instance of the Census API...
Census API instance created:


census.core.Census

## Census API Aliases

In [59]:
# Load census aliases
# TODO: Add google colab-specific code to load the census aliases
if dev: print("Loading census aliases...")
with open('data/census/census_aliases.json') as f:
    census_aliases = json.load(f)

# Display the census aliases
# display(census_aliases)

def printAliases():
  # Show the keys in the census aliases
  alias_count = 1
  print("Census aliases:")
  print("-"*30)
  for key, value in census_aliases.items():
      description = value["description"]
      column_name = value["variable"]
      title = value["text"]
      unit = value["unit"]
      print(f"{alias_count:03d} | {title.capitalize()}: {description}")
      print(f"    | Census column name: {column_name}")
      print(f"    | Unit: {unit}")
      print("-"*30)
      alias_count+=1

# Saving the reference of the standard output
original_stdout = sys.stdout 	

try:
  # Save the census aliases to a file
  if dev: print("Saving census aliases to file...")
  with open('census_aliases.txt', 'w') as f:
      sys.stdout = f
      printAliases()
      # Reset the standard output
      sys.stdout = original_stdout
  print("Census aliases saved to file successfully!")
except:
  print("Error saving census aliases to file")


Loading census aliases...
Saving census aliases to file...
Census aliases saved to file successfully!


## FIPS Codes

### Functions for obtaining county FIPS codes

In [60]:
# Define funtions to get FIPS codes for US counties
# https://gist.github.com/cjwinchester/a8ff5dee9c07d161bdf4
def getCounties():
    "Function to return a dict of FIPS codes (keys) of U.S. counties (values)"
    d = {}
    r = requests.get("http://www2.census.gov/geo/docs/reference/codes/files/national_county.txt")
    reader = csv.reader(r.text.splitlines(), delimiter=',')    
    for line in reader:
        d[line[1] + line[2]] = line[3].replace(" County","")    
    return d

def getCountyAdj():
    "Return a list of dicts where each dict has a county FIPS code (key) and a list of FIPS codes of the adjacent counties, not including that county (value)"
    adj = requests.get("http://www2.census.gov/geo/docs/reference/county_adjacency.txt")
    adj_data = adj.text.encode("utf-8")
    reader = csv.reader(adj_data.splitlines(), delimiter='\t')
    ls = []
    d = {}
    countyfips = ""
    for row in reader:
        if row[1] and row[1] != "":
            if d:
                ls.append(d)
            d = {}
            countyfips = row[1]
            d[countyfips] = []
            "Grab the record on the same line"
            try:
                st = row[3]
                if st != countyfips:
                    d[countyfips].append(st)
            except:
                pass
        else:
            "Grab the rest of the records"
            if row[3] and row[3] != "":
                st = row[3]
                if st != countyfips:
                    d[countyfips].append(st)


### Obtaining state FIPS codes

In [61]:
# Get the FIPS codes for Colorado
try:
  if dev: print("Getting FIPS codes for Colorado...")
  colorado_fips = states.CO.fips
  state_fips = colorado_fips
  print(f"Colorado FIPS code: {state_fips}")
except:
  print("Error getting FIPS codes for Colorado")


Getting FIPS codes for Colorado...
Colorado FIPS code: 08


### Obtaining all US county FIPS codes

In [62]:
# Store county FIPS codes in memory
try:
  if dev: print("Storing county FIPS codes in memory...")
  county_fips = getCounties()
  if dev: print("All County FIPS codes:")
  if dev: display(county_fips)
except:
  print("Error storing county FIPS codes in memory")
  

Storing county FIPS codes in memory...
All County FIPS codes:


{'01001': 'Autauga',
 '01003': 'Baldwin',
 '01005': 'Barbour',
 '01007': 'Bibb',
 '01009': 'Blount',
 '01011': 'Bullock',
 '01013': 'Butler',
 '01015': 'Calhoun',
 '01017': 'Chambers',
 '01019': 'Cherokee',
 '01021': 'Chilton',
 '01023': 'Choctaw',
 '01025': 'Clarke',
 '01027': 'Clay',
 '01029': 'Cleburne',
 '01031': 'Coffee',
 '01033': 'Colbert',
 '01035': 'Conecuh',
 '01037': 'Coosa',
 '01039': 'Covington',
 '01041': 'Crenshaw',
 '01043': 'Cullman',
 '01045': 'Dale',
 '01047': 'Dallas',
 '01049': 'DeKalb',
 '01051': 'Elmore',
 '01053': 'Escambia',
 '01055': 'Etowah',
 '01057': 'Fayette',
 '01059': 'Franklin',
 '01061': 'Geneva',
 '01063': 'Greene',
 '01065': 'Hale',
 '01067': 'Henry',
 '01069': 'Houston',
 '01071': 'Jackson',
 '01073': 'Jefferson',
 '01075': 'Lamar',
 '01077': 'Lauderdale',
 '01079': 'Lawrence',
 '01081': 'Lee',
 '01083': 'Limestone',
 '01085': 'Lowndes',
 '01087': 'Macon',
 '01089': 'Madison',
 '01091': 'Marengo',
 '01093': 'Marion',
 '01095': 'Marshall',
 '01097': 

### Obtaining FIPS codes for all counties in Colorado

In [63]:
# Filter out only Colorado county FIPS codes
try:
  if dev: print("Filtering out only Colorado county FIPS codes...")
  colorado_county_fips = {key: value for key, value in county_fips.items() if key.startswith(state_fips)}
  if dev: print("Colorado County FIPS codes:")
  if dev: display(colorado_county_fips)
  print(f"Number of Colorado counties: {len(colorado_county_fips)}")
except:
  print("Error filtering out only Colorado county FIPS codes")


Filtering out only Colorado county FIPS codes...
Colorado County FIPS codes:


{'08001': 'Adams',
 '08003': 'Alamosa',
 '08005': 'Arapahoe',
 '08007': 'Archuleta',
 '08009': 'Baca',
 '08011': 'Bent',
 '08013': 'Boulder',
 '08014': 'Broomfield',
 '08015': 'Chaffee',
 '08017': 'Cheyenne',
 '08019': 'Clear Creek',
 '08021': 'Conejos',
 '08023': 'Costilla',
 '08025': 'Crowley',
 '08027': 'Custer',
 '08029': 'Delta',
 '08031': 'Denver',
 '08033': 'Dolores',
 '08035': 'Douglas',
 '08037': 'Eagle',
 '08039': 'Elbert',
 '08041': 'El Paso',
 '08043': 'Fremont',
 '08045': 'Garfield',
 '08047': 'Gilpin',
 '08049': 'Grand',
 '08051': 'Gunnison',
 '08053': 'Hinsdale',
 '08055': 'Huerfano',
 '08057': 'Jackson',
 '08059': 'Jefferson',
 '08061': 'Kiowa',
 '08063': 'Kit Carson',
 '08065': 'Lake',
 '08067': 'La Plata',
 '08069': 'Larimer',
 '08071': 'Las Animas',
 '08073': 'Lincoln',
 '08075': 'Logan',
 '08077': 'Mesa',
 '08079': 'Mineral',
 '08081': 'Moffat',
 '08083': 'Montezuma',
 '08085': 'Montrose',
 '08087': 'Morgan',
 '08089': 'Otero',
 '08091': 'Ouray',
 '08093': 'Park',
 

Number of Colorado counties: 64


### Obtaining FIPS code for a specific county in Colorado

In [ ]:
# Prompt for county to look up and retrieve fip
# county_name = input("Enter county name to look up the FIPS code: ")
try:
  if dev: print("Looking up the FIPS code for Denver county...")
  county_name = "Denver"
  state_county_fip = list(county_fips.keys()) [list(county_fips.values()).index(county_name)]
  county_fip = state_county_fip[2:]
  print(f"The FIPS code for {county_name} is {county_fip}")
except:
  print(f"Error looking up the FIPS code for {county_name} county")


## Census Data

### Getting Raw Census Data

In [64]:
# Run Census Search to retrieve data on all zip codes
census_start_year = 2011
census_end_year = 2021
census_years = range(census_start_year, census_end_year + 1)
data_retrieval_output = sys.stdout

# Create a DataFrame to hold the census data with an empty row so we can append data to it
raw_census_df = pd.DataFrame({
  "NAME": "",
  "B19013_001E": "",
  "B19301_001E": "",
  "B23025_002E": "",
  "B23025_007E": "",
  "B17001_002E": "",
  "B01002_001E": "",
  "B01003_001E": "",
  "B25035_001E": "",
  "B25058_001E": "",
  "B25064_001E": "",
  "B25077_001E": "",
  "B25088_002E": "",
  "Year": ""
}, index=[0])

# Loop through the years and retrieve data for each year
try:
  for county_fip in colorado_county_fips.keys():
    for year in census_years:
      if dev: print(f"Retrieving data for {year} {colorado_county_fips[county_fip]} County...")
      year_data = c.acs5.state_county(
        (
          "NAME",
          "B19013_001E",
          "B19301_001E",
          "B23025_002E",
          "B23025_007E",
          "B17001_002E",
          "B01002_001E",
          "B01003_001E",
          "B25035_001E",
          "B25058_001E",
          "B25064_001E",
          "B25077_001E",
          "B25088_002E"
        ),
        colorado_fips,
        county_fip[2:],
        year=year
      )

      # Display the retrieved data
      if dev: print(f"Data retrieved for {year} {colorado_county_fips[county_fip]} County.")
      if dev: display(year_data)

      # Convert to DataFrame
      if dev: print("Converting data to DataFrame...")
      year_df = pd.DataFrame(year_data)

      # Add the data's year to a Year column
      if dev: print(f"Adding Year column to {year} {colorado_county_fips[county_fip]} County...")
      year_df["Year"] = year

      # Add the data to the combined DataFrame
      if dev: print("Adding data to the combined DataFrame...")
      raw_census_df = pd.concat([raw_census_df, year_df], ignore_index=True)

      # Wait for 5 seconds before making the next request
      if dev: print("Waiting for 5 seconds before making the next request...")
      if dev: print("-"*50)
      # TODO: Show percentage of data retrieval completion
      clear_output(wait=True)
      time.sleep(5)
    print(f"Data retrieval of {colorado_county_fips[county_fip]} County complete.")
  print(f"Data retrieval of all Colorado counties complete.")
except Exception as e:
  print("Error retrieving data from the U.S. Census API")
  print(e)


Retrieving data for 2018 El Paso County...
Data retrieved for 2018 El Paso County.


[{'NAME': 'El Paso County, Colorado',
  'B19013_001E': 65334.0,
  'B19301_001E': 32348.0,
  'B23025_002E': 364071.0,
  'B23025_007E': 173477.0,
  'B17001_002E': 72948.0,
  'B01002_001E': 34.1,
  'B01003_001E': 688153.0,
  'B25035_001E': 1986.0,
  'B25058_001E': 967.0,
  'B25064_001E': 1119.0,
  'B25077_001E': 254300.0,
  'B25088_002E': 1554.0,
  'state': '08',
  'county': '041'}]

Converting data to DataFrame...
Adding Year column to 2018 El Paso County...
Adding data to the combined DataFrame...
Waiting for 5 seconds before making the next request...
--------------------------------------------------


### Saving Raw Census Data to CSV

In [ ]:
# Save the raw census data to a CSV file
try:
  if dev: print("Saving raw census data to a CSV file...")
  raw_census_df.to_csv("data/census/raw_census_data.csv", index=False)
  print("Raw census data saved to file successfully!")
except:
  print("Error saving raw census data to file")
  

### Loading Raw Census Data from CSV

In [ ]:
# Load the raw census data from the CSV file
try:
  if dev: print("Loading raw census data from the CSV file...")
  raw_census_df = pd.read_csv("data/census/raw_census_data.csv")
  if dev: display(raw_census_df.head())
  print("Raw census data loaded from file successfully!")
except:
  print("Error loading raw census data from file, please make sure the file exists.")


### Cleaning Census Data

#### Dropping rows with missing values

In [ ]:
# Drop the empty row from the combined DataFrame
try:
  if dev: print("Dropping the empty row from the combined DataFrame...")
  raw_census_df.dropna(inplace=True)
  if dev: print("Combined DataFrame:")
  display(raw_census_df.head())
except:
  print("Error dropping empty row from the combined DataFrame")


#### Index fixing and Sorting

In [ ]:
# Copy the DataFrame to a new DataFrame
if dev: print("Copying the DataFrame to a new DataFrame...")
census_df = raw_census_df.copy()
# Reset the index of the DataFrame
if dev: print("Resetting the index of the DataFrame...")
census_df.reset_index(inplace=True, drop=True)
# Sort the DataFrame by Name and Year
if dev: print("Sorting the DataFrame by Name and Year...")
census_df.sort_values(by=["NAME", "Year"], inplace=True)
# Display the DataFrame
if dev: print("New DataFrame:")
if dev: display(census_df.head())


#### Column fixing

In [ ]:
# Convert the Year column to an integer
try:
  if dev: print("Converting the Year column to an integer...")
  census_df["Year"] = census_df["Year"].astype(int)
  if dev: print("Year column converted to an integer successfully!")
except:
  print("Error converting the Year column to an integer")


In [ ]:
# Column renaming
if dev: print("Renaming columns...")
census_df.rename(
  columns = {
    "NAME": "County",
    "B19013_001E": "Median Household Income",
    "B19301_001E": "Per Capita Income",
    "B23025_002E": "Number of Employed (16+)",
    "B23025_007E": "Number of Unemployed (16+)",
    "B17001_002E": "Poverty Count",
    "B01002_001E": "Median Age",
    "B01003_001E": "Total Population",
    "B25035_001E": "Average Year Home Built",
    "B25058_001E": "Median Contract Rent",
    "B25064_001E": "Average Cost of Rent and Utilities",
    "B25077_001E": "Average Value of Owner-occupied Home",
    "B25088_002E": "Average Monthly Housing Expense with Mortgage"
  },
  inplace=True
)

# Add a Poverty Rate column (Poverty Count / Population)
if dev: print("Adding a Poverty Rate column...")
census_df["Poverty Rate"] = 100 * census_df["Poverty Count"].astype(int) / census_df["Population"].astype(int)


In [ ]:
# Configure the final DataFrame
if dev: print("Configuring the DataFrame...")
census_df = census_df[
  [
    "County",
    "Year",
    "Median Household Income",
    "Per Capita Income",
    "Number of Employed (16+)",
    "Number of Unemployed (16+)",
    "Poverty Count",
    "Median Age",
    "Total Population",
    "Average Year Home Built",
    "Median Contract Rent",
    "Average Cost of Rent and Utilities",
    "Average Value of Owner-occupied Home",
    "Average Monthly Housing Expense with Mortgage"
  ]
]

# Display DataFrame length and sample data
if dev: print(f"Number of rows in the DataFrame: {len(census_df)}")
if dev: print("Combined county DataFrame preview:")
if dev: display(census_df.head())


#### Saving Cleaned Census Data to CSV

In [ ]:
# Save the final census data to a CSV file
try:
  if dev: print("Saving final census data to a CSV file...")
  census_df.to_csv("data/census/clean_census_data.csv", index=False)
  print("Final census data saved to file successfully!")
except:
  print("Error saving final census data to file")


#### Loading Cleaned Census Data from CSV

In [ ]:
# Load the final census data from the CSV file
try:
  if dev: print("Loading final census data from the CSV file...")
  census_df = pd.read_csv("data/census/clean_census_data.csv")
  if dev: display(census_df.head())
  print("Final census data loaded from file successfully!")
except:
  print("Error loading final census data from file, please make sure the file exists.")


#### Displaying Cleaned Census Data

In [ ]:
# Display the first and last 30 rows of the DataFrame
print("First 30 rows of the DataFrame:")
display(census_df.head(30))

print("Last 30 rows of the DataFrame:")
display(census_df.tail(30))


# School Code

In [ ]:
# Import School data into a DataFrame
if dev: print("Loading combined school data...")
raw_school_data_df = pd.read_excel("data/schools/combined_2011-21.xlsx")

print("School data loaded successfully!")


In [ ]:
# Copy the DataFrame to a new DataFrame
if dev: print("Copying the DataFrame to a new DataFrame...")
school_data_df = raw_school_data_df.copy()
# Sort the DataFrame by Name and Year
if dev: print("Sorting the DataFrame by County and Year...")
school_data_df.sort_values(by=["County", "Year"], inplace=True)
# Reset the index of the DataFrame
school_data_df.reset_index(inplace=True, drop=True)
# Display the DataFrame
if dev: print("New DataFrame:")
if dev: display(school_data_df.head())


In [ ]:
# Display the first and last 30 rows of the DataFrame
print("School data:")
display(school_data_df.head(30))
display(school_data_df.tail(30))


# HIC Data

In [ ]:
# Import HIC data into a DataFrame
if dev: print("Loading HIC data...")
raw_hic_data_df = pd.read_excel("data/hic/HIC_CO_2007-2023.xlsx")

print("HIC data loaded successfully!")


In [ ]:
# Display the first and last 30 rows of the DataFrame
print("HIC data:")
display(raw_hic_data_df.head(30))
display(raw_hic_data_df.tail(30))


# PIT Data

In [ ]:
# Import PIT data into a DataFrame
if dev: print("Loading PIT data...")
raw_pit_data_df = pd.read_excel("data/pit/PIT_CO_2007-2023.xlsx")

print("PIT data loaded successfully!")


In [ ]:
# Display the first and last 30 rows of the DataFrame
print("PIT data:")
display(raw_pit_data_df.head(30))
display(raw_pit_data_df.tail(30))


# SNAP Data

In [ ]:
# Import SNAP data into a DataFrame
if dev: print("Loading SNAP data...")
raw_snap_data_df = pd.read_csv("data/snap/SNAP_CO_2010-2022.csv")

print("SNAP data loaded successfully!")


In [ ]:
# Display the first and last 30 rows of the DataFrame
print("SNAP data:")
display(raw_snap_data_df.head(30))
display(raw_snap_data_df.tail(30))


# SSS Data

In [ ]:
# Import SSS data into a DataFrame
if dev: print("Loading SSS data...")
raw_sss_data_df = pd.read_excel("data/sss/SSS_CO_2011-2022.xlsx")

print("SSS data loaded successfully!")


In [ ]:
# Display the first and last 30 rows of the DataFrame
print("SSS data:")
display(raw_sss_data_df.head(30))
display(raw_sss_data_df.tail(30))
